<a href="https://colab.research.google.com/github/kiranmahara9600/AirbnbNYC_2019_Analysis/blob/main/Airbnb_Bookings_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***Since 2008, guests and hosts have used Airbnb to expand on traveling possibilities and present a more unique, personalized way of experiencing the world. Today, Airbnb became one of a kind service that is used and recognized by the whole world. Data analysis on millions of listings provided through Airbnb is a crucial factor for the company. These millions of listings generate a lot of data - data that can be analyzed and used for security, business decisions, understanding of customers' and providers' (hosts) behavior and performance on the platform, guiding marketing initiatives, implementation of innovative additional services and much more.***

# ***This dataset has around 49,000 observations in it with 16 columns and it is a mix between categorical and numeric values.***

***Explore and analyze the data to discover key understandings (not limited to these) such as :***

*What can we learn about different hosts and areas?*

*What can we learn from predictions? (ex: locations, prices, reviews, etc)*

*Which hosts are the busiest and why?*

*Is there any noticeable difference of traffic among different areas and what could be the reason for it?*

In [ ]:
#importing all the necessary modules/libraries required during the analysis
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#mounting google drive in colab
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#reading airbnb NYC 2019 csv data and storing it in airbnbNY_data dataframe
file_path='/content/drive/MyDrive/Datascience/Datasets/Airbnb NYC 2019.csv'
airbnbNY_data=pd.read_csv(file_path)

# ***Part 1: Basic datapreprocessing and Cleaning***

In [ ]:
#----------------------Block 1----------------------------------

#checking for no. of observations and features in our dataset
airbnbNY_data.shape

*We have a listings of 48895 observations with 16 features*

In [ ]:
#----------------------Block 2----------------------------------

# visualising first 5 observations from our dataset 
airbnbNY_data.head()

 ****Data Dictionary****

*1. ID - ID is just an unique identifier of each row.*

*2. Name - It is a rental name which user sees while booking(hotel name).*

*3. Host ID - Every host on Airbnb gets an unique host id, host id here represents that id only.*

*4. Host name - Name of the host.*

*5. Neighbourhood_group - Whole New York is divied into 5 neighbourhood_group(boroughs) regionally. It shows in which borough particular listing is located.*

*6. Neighbourhood - Each borough is further sub divided into neighbourhoods. This column shows the neighbourhood of particular listing.*

*7. Latitude and longitude - These shows the geographical positions of a rental.*

*8. Room_Type -  Which room type a rental is offering.*

*9. Price - Price per night that a rental is charging.*

*10. Minimum Nights - For this much of nights minimally rental can be booked.*

*11. Number of reviews - Number of reviews a rental has got till now.*

*12. Review Per month - An average no. of reviews that a rental got per month.*

*13. Last review - Date on which a rental got their last review.*

*14. calculated_host_listings_count -  It is the no. of rentals a host is hosting in the dataset.*

*15. Availability 365 - It is the no. of days for which a rental is available to book.*

*NOTE - I have used the word rentals, listings and hostings interchangably here. All of these means the same.*

In [ ]:
#----------------------Block 3----------------------------------

#getting basic information about dataset
airbnbNY_data.info()

*Outcomes*

*1. Dataype of each columns*

*2. Memory usage due to dataset*

*3. Number of non null values in each columns*


In [ ]:
#----------------------Block 4----------------------------------

# getting count of each null values in each column
airbnbNY_data.isnull().sum()

*1. Although Name and Host name has null values but it won't be of significant use in our analysis, we won't be caring much about it.*

*2. Last review and review per month both have same number of null values. Actual reason is they are interdependent. Last_review includes a date of last review of specific listing. If it is null that means listing hasn't got any reviews yet and for the same reason review per month is also null.*

*3. If you look for the observations having review_per_month and last_review as null values, logiclly no. of reviews value should be zero there. So, lets verify this.* 



In [ ]:
#----------------------Block 5----------------------------------

# checking for number_of_review unique values where we have null values for last_review and review_per_month 
airbnbNY_data[airbnbNY_data['last_review'].isnull() & airbnbNY_data['reviews_per_month'].isnull()]['number_of_reviews'].value_counts()

*We will delete the last review column from dataframe as that is not much useful in terms of further data analysis make reviews_per_ month  0 where reviews_per_month is Null.*

In [ ]:
#----------------------Block 6----------------------------------

airbnbNY_data.drop(['last_review'],axis = 1, inplace = True)   # dropping last review column
airbnbNY_data['reviews_per_month'].fillna(0,inplace=True)    #filling null values with 0 in reviews_per_month column

*Name of hotel is also not of a significant use to us so we'll drop that as well*

In [ ]:
#----------------------Block 7----------------------------------

airbnbNY_data.drop(['name'],axis = 1, inplace = True)   # dropping name column
airbnbNY_data.drop(['host_name'],axis = 1, inplace = True)   # dropping host_name column

In [ ]:
#----------------------Block 8----------------------------------

#null values check
airbnbNY_data.isnull().sum()

*We have treated null values and dropped all unneccesary columns as well .*

In [ ]:
#----------------------Block 9----------------------------------

#checking for descriptive summary of data
airbnbNY_data.drop(['id','host_id','latitude','longitude'], axis =1).describe()

*Descriptive table tells us that except availability 365, all other features has some serious outliers.*

*As a part of this project we are only performing EDA .So there might not be a need of outlier treatment but in some specific cases outliers affects our EDA task as well. So we'll look for temporary treatment at that point.*


# ***Part 2: Feature analysis and engineering***

##***1 .Visualising and analysing about different areas(neighbourhood and boroughs)***

In [ ]:
#----------------------Block 10----------------------------------

# unique categories in neighbourhood_group
airbnbNY_data['neighbourhood_group'].unique()

*1. There are total of 5 neighbourhood groups.*


*2. In New York whole New York city has been divided in these(neighbourhood_group) five adminstrative region and these are called borough there. So we will rename the neighbourhood_group columns to borough for better comprehensiveness.*


In [ ]:
#----------------------Block 11----------------------------------

airbnbNY_data.rename(columns={'neighbourhood_group':'borough'}, inplace=True) # renaming neighbourhood group column to borough

In [ ]:
#----------------------Block 12----------------------------------


airbnbNY_data.head()  # data check for column rename 

###**Let's look into how hosting is divided among these boroughs**

In [ ]:
#----------------------Block 13----------------------------------

#checking for the observations under each borough
borough_hositing = airbnbNY_data['borough'].value_counts().sort_values(ascending=False)
borough_hositing

In [ ]:
#----------------------Block 14----------------------------------

#plotting the above found details
fig,axes= plt.subplots(1,2,figsize=(16,6))  #defining figure and initialising axes 
borough_hositing.plot.pie(autopct = '%1.2f%%', ax = axes[0]) #pie plot in axes[0]
axes[0].set_title('Percentage split of hostings \n among each borough')  # title for pie plot
axes[0].legend()  #to get legends
borough_hositing.plot.bar(ax = axes[1],color=['blue','orange','green','red','purple'])  #bar plot in axes[1]
axes[1].set_title('Number of hostings among different boroughs')  # title for bar plot
axes[1].set(ylabel='No. of hostings', xlabel='Boroughs')   # labelling y and x axis
fig.tight_layout() #for creating proper spacing and arrangement among different figures on canvas

*From above charts we can see that major no. of hostings are from Brooklyn and Manhattan from this listing of airbnb New York city whereas Staten Island and Bronx are way too less populous in terms of hostings.*

*Actually Manhattan is one of  the most attracted tourism borough in New York called to be as a heart of New York city and Brooklyn follows after Manhattan and also it is most populous borough that is one of the strong reason for Brooklyn and Manhattan to be dominating in terms of hostings.* 


###**Checking for some of the top neighbourhoods and to which borough they belong**




*Note : We are qualifying neighbourhood as top on the basis of number of hostings.*



In [ ]:
#----------------------Block 15----------------------------------

#grouping observation on borough and then on neighbourhood and then picking up their count to find no. of hostings in different neighbourhoods
top_20_neighbourhood = airbnbNY_data.groupby(['borough','neighbourhood']).agg({'id':
                                            'count'}).sort_values(by='id',ascending=False)[0:20].reset_index()  #picking top 20 neighbourhoods
top_20_neighbourhood.rename(columns={'id':'num of hostings'},inplace=True)  #renaming column 'id' to 'num of hostings'
top_20_neighbourhood

In [ ]:
#----------------------Block 16----------------------------------

#visualising Block 15
plt.figure(figsize=(15,7))  #defining figure
from matplotlib.lines import Line2D  #importing Line2D
borough_dict = {} #initialising an empty dictionary to store the colour representation of boroughs
unique_borough = set(list(top_20_neighbourhood['borough']))  #storing unique boroughs of top_20_neighbourhood in unique_borough
borough_list = list(top_20_neighbourhood['borough'])  #storing boroughs of top_20_neighbourhood in the form of list 
color_list=[]  #initialising an empty list for storing the colour of each bar in bar plot
color=['darkslategray','teal','darkturquoise']
index=0
for borough in unique_borough:
  borough_dict[borough] =color[index]  #assigning unique colour that represents each borough
  index=index+1
for borough in borough_list:   
       color_list.append(borough_dict[borough])  #storing the colour of each bar of bar plot in color_list
plt.bar(top_20_neighbourhood['neighbourhood'],top_20_neighbourhood['num of hostings'],color=color_list)  #plotting a bar plot
#creating a custom labels for a graph
labels = list(borough_dict.keys())
custom_lines = [Line2D([1,2,5,5], [1,2,5,5], color='darkslategray', lw=4),
                Line2D([1,2,5,5], [1,2,5,5], color='teal', lw=4),
                Line2D([1,2,5,5], [1,2,5,5], color='darkturquoise', lw=4)]
plt.xticks(rotation = 90)  #to rotate x-ticks by 90 degree
plt.legend(custom_lines,labels)  #to get legends in graph
plt.xlabel('neighbourhood',fontsize=13)
plt.ylabel('Number of hosts',fontsize=13)
plt.title("Top 20 neighbourhoods in terms of number of hostings",fontsize=15)
plt.show()

*From the above graph we can see that :*

* Among top 20 neighbourhoods most of these neighbourhood belongs to Manhattan borough.
*Although If looking for top 3 densely hosted neighbourhood top 2 belongs to Brooklyn.

In [ ]:
len(airbnbNY_data[airbnbNY_data['calculated_host_listings_count']<8])

###**Neighbourhoods each borough have**

In [ ]:
#----------------------Block 17----------------------------------

fig = plt.figure(figsize=(10,6))  #defining figure
ngbrhood = airbnbNY_data.groupby('borough')['neighbourhood'].nunique().sort_values(ascending=False)  #number of unique neighbourhoods in each borough
ngbrhood.plot.bar(color='orange')  #bar plot representing no. of neighbourhoods in each borough
plt.ylabel('No. of neighbourhoods')
plt.title('Number of neighbourhoods in each borough',fontsize=15)
for ind, val in enumerate(ngbrhood):
  plt.text(x = ind, y =val+1, s = str(val))  #to get text over bars

*Although Mahnhattan and Brooklyn have most no. of hostings but in terms of neighbourhood they are quite back.*

*Reason for it can be Manhattan and Brooklyn have most tourist places and many hostings has been bunched around under same neigbourhood. Thus having most hostings without having much of neighbourhoods.*

###**Lets look for the price distribution in each boroughs**

In [ ]:
#----------------------Block 18----------------------------------

boroughs = set(list(airbnbNY_data['borough']))  #storing distinct boroughs
for borough in boroughs:
  fig = plt.figure(figsize=(5,5))
  axs = fig.gca()  #gca--> get current axis
  price_dist = list(airbnbNY_data[airbnbNY_data['borough']== borough]['price'])  #list of price for each borough
  price_dist = [price for price in price_dist if ((price < 600) and (price>0))]  #Limiting price value to 600 to avoid outliers that limits our visualisation
  axs.set_title(borough)
  sns.distplot(price_dist, ax=axs,color='green')  #price distribution across each borough

*Note as told earlier, as needed here, we have done an outlier treatment.*

*Even after removing outliers still the data is bit skewed. We can further decrease the threshold of price also.*

*Starting price ranges are nearly same for each boroughs and they are starting from as low as to somewhere 10 dollars and most of them are ending at 200 with few rentals having price greater than 200.In which manhattan has max number of rentals with price greater than 200*

*Note : Prices are heavily influenced by the outliers .I have set an threshold of price 600 in order to avoid outlier so that we can visualize the range better.*

###**Let's check how does an average price varies in each borough**

In [ ]:
#----------------------Block 19----------------------------------

fig= plt.figure(figsize=(10,6))
axes = fig.gca()
avgprice_df = airbnbNY_data[airbnbNY_data['price']<600]  #creating sub dataframe with price less than 600
#grouping by borough and averaging out by price to find average price across each borough
avgprice_borough = avgprice_df.groupby('borough').agg({'price':'mean'}).sort_values(by='price',ascending = False)
avgprice_borough.plot.bar(ax = axes)  #bar plot represeting average price across each borough
plt.title('Average Pricing in each borough',fontsize=15)
avgprice = list(np.around(avgprice_borough.values.reshape(5,))) #rounding up and storing average price of each borough in a list
axes.set_ylabel('average price')
for ind, val in enumerate(avgprice):
  axes.text(x = ind, y =val+3, s = str(val))  #to put text over bars

* Manhattan is one of the borough that charges quite high for hostings followed by Brooklyn.

* Manhattan include most of the tourist places including Times Squares, etc that can be the reason of this price hikes.

*Note - Please note that I have taking a threshold of 600 here as well because we know that average is heavily affected by outliers* 


###**Let's check the same in terms of price for each neighbourhood**

In [ ]:
#----------------------Block 20----------------------------------

#Seperating out the top 10 neighbourhoods with highest average charges
avgprice_neighbourhood = airbnbNY_data.groupby(['borough','neighbourhood']).agg({'price':'mean'}).reset_index().sort_values(by='price',ascending=False)[0:10]
plt.figure(figsize=(15,6))
borough_dict = {}   #dictionary to store unique borough and color label associated with it 
unique_borough = set(list(avgprice_neighbourhood['borough']))  #seperating out the unique borough names from sub dataframe
borough_list = list(avgprice_neighbourhood['borough'])  #listing out all observations boroughs from sub dataframe
color_list=[]  #list to store the color associated with bar plots
color=['bisque','cyan','pink','lavender','red']
index=0 
for borough in unique_borough:
  borough_dict[borough] =color[index]  #assigning key value pair in borough_dict with keys as borough of unique_borough and values as colours of color list 
  index=index+1
for borough in borough_list:   
       color_list.append(borough_dict[borough])  
plt.bar(avgprice_neighbourhood['neighbourhood'],avgprice_neighbourhood['price'],color=color_list)  #bar plot representing top 10 average price neighbourhoods
labels = list(borough_dict.keys())
custom_lines = [Line2D([0], [0], color='bisque', lw=4),  # custom limes for label of bar graph 
                Line2D([0], [0], color='cyan', lw=4),
                Line2D([0], [0], color='pink', lw=4),
                Line2D([0], [0], color='lavender', lw=4),
                Line2D([0], [0], color='red', lw=4)]
plt.xticks(rotation = 90)
plt.legend(custom_lines,labels)
plt.xlabel('neighbourhood',fontsize=13)
plt.ylabel('Average Price',fontsize=13)
plt.title("10 Neighbourhoods having highest average prices",fontsize=15)
plt.show()

*Staten Islands neighbourhoods seems to have some of the top chargable rentals and Manhattan follows them. Although from our previous analysis it seems that Manhattan is one of the expensive borough but still We have to investigate it a bit further as this might be the result of strong outliers as well. So lets visualize box plots for these 10 neighbourhoods to know if they actually have a higher prices or its just some strong outliers.*

In [ ]:
#----------------------Block 21----------------------------------

# nghbrhood function to segregate the dataframe only for top 10 neighbourhoods
def nghbrhood(col):
  if col in neighbourhood_lst:
    return True
  else:
    return False

In [ ]:
#----------------------Block 22----------------------------------

neighbourhood_lst = list(avgprice_neighbourhood['neighbourhood'])  #storing list of top 10 neighbourhoods
plt.figure(figsize=(15,8))
df_nghbrhoodprice = airbnbNY_data[airbnbNY_data['neighbourhood'].apply(nghbrhood)]  #applying above defined function 
sns.boxplot(x='neighbourhood',y='price', data = df_nghbrhoodprice )  #boxplot showing price distribution across each of top 10 neighbourhoods
plt.title('Price distribution in top 10 neighbourhoods',fontsize=15)
plt.show()

*So as thought it seems that Fort Wadsworth and Woodrow doesn't have multiple entries they both only have one listing, Fort Wadsworth with price nearby 800 and Woodsrow having a price nearby 700.*

*Whereas Tribeca, Flatiron district and battery park city is heavily influenced by the outliers.*

*Prince's bay and River Dales are the ones that doesn't seems to be affected by the outlier and in general have higher prices.*

*Lets see if we try with the threshold price of 600 how does result varies.*

In [ ]:
#----------------------Block 23----------------------------------

#Seperating out the top 10 neighbourhoods with highest average charges with price threshold 600
avgprice_neighbourhood = avgprice_df.groupby(['borough','neighbourhood']).agg({'price':'mean'}).reset_index().sort_values(by='price',
ascending=False)[0:10]
plt.figure(figsize=(15,6))
borough_dict = {}   #dictionary to store unique borough and color label associated with it 
unique_borough = set(list(avgprice_neighbourhood['borough']))  #seperating out the unique borough names from sub dataframe
borough_list = list(avgprice_neighbourhood['borough'])  #listing out all observations boroughs from sub dataframe
color_list=[]  #list to store the color associated with bar plots
color=['bisque','cyan','pink','blue','red']
index=0
for borough in unique_borough:
  borough_dict[borough] =color[index]  #assigning key value pair in borough_dict with keys as borough of unique_borough and values as colours of color list 
  index=index+1
for borough in borough_list:   
       color_list.append(borough_dict[borough])
plt.bar(avgprice_neighbourhood['neighbourhood'],avgprice_neighbourhood['price'],color=color_list)
labels = list(borough_dict.keys())
custom_lines = [Line2D([0], [0], color='bisque', lw=4),  # custom limes for label of bar graph 
                Line2D([0], [0], color='cyan', lw=4),
                Line2D([0], [0], color='pink', lw=4),
                Line2D([0], [0], color='blue', lw=4),
                Line2D([0], [0], color='red', lw=4)]
plt.xticks(rotation = 90)
plt.legend(custom_lines,labels)
plt.xlabel('neighbourhood',fontsize=13)
plt.ylabel('Average Price',fontsize=13)
plt.title("10 Neighbourhoods having highest average prices",fontsize=15)
plt.show()

In [ ]:
#----------------------Block 24----------------------------------

neighbourhood_lst = list(avgprice_neighbourhood['neighbourhood'])  #storing list of top 10 neighbourhoods
plt.figure(figsize=(15,8))
df_nghbrhoodprice = avgprice_df[avgprice_df['neighbourhood'].apply(nghbrhood)]  #applying defined function in Block21
sns.boxplot(x='neighbourhood',y='price', hue='borough',data = df_nghbrhoodprice)  #boxplot showing price distribution across each of top 10 neighbourhoods
plt.title('Price distribution in top 10 neighbourhoods',fontsize=15) #adding title to the plot
plt.show()

*As we have changed a price threshold to 600 picture changes pretty much thus these neighhbourhood prices are influenced by outliers. You can also try changing multiple threshold and can visualize how the graph and neighbourhood changes* 

*We can see, most of the top 10 neighbourhoods belongs to Manhattan*

###**We have played pretty much along with area and collected an info :**


*1. Among all 5 boroughs Manhattans and Brooklyn are the ones who have most rentals. Manhattan is the most famous tourism borough and that is one of the reasons for most rentals.*

*2. Although Manhattan dominates in no. of hostings but neighbourhoods of Brooklyn dominates on first 2 positions in among top 20 neighbourhoods. Still if we quantify the result than we see we have most number of Manhattan neighbourhoods tooks a place among top 20 neighbourhood.*

*3. Williamsburg is a hip neighbourhood in Brooklyn that is famous for trendy cafes and buzzy restaurants. Street art brightens residential thoroughfares and repurposed factories, while the waterfront provides stunning views of Manhattan and seasonal venues for outdoor concerts and food markets.*

*4.Number of neighbourhood is highest in Queens and lowest in Manhattan still Manhattan has most no. of rentals.*

*5. Due to this tourism factor average price of rentals is higher in Manhattan and after Manhattan Brooklyn follows it.*

*6. Price data contains too many outliers, mostly the price of hotel saturates at max near 200-300 Dollar but we have some rentals charging as high as 10000 Dollar. Thus creating a bias in average prices. We have tried to visualize the price data among different neighbourhoods and in that case also after outlier treatment out of top higest pricing neighbourhood 7 belongs to Manhattan.*


## ***2. Visualising and analysing about different room types***

In [ ]:
#----------------------Block 25----------------------------------

#unique room_type values 
airbnbNY_data['room_type'].unique()

###**Lets check the room type feature**

In [ ]:
#----------------------Block 26----------------------------------

plt.rcParams['figure.figsize']=(10,6)  #defining figure
airbnbNY_data['room_type'].value_counts().plot.bar()  #bar plot representing counts of each room type
plt.title('Number of each roomtype rentals in a dataset',fontsize=15)
plt.xlabel('Room_type',fontsize=13)
plt.ylabel('counts',fontsize=13)
plt.show()

*Mostly rentals are there for entire home or private rooms and just very few in case of shared room. Shared room might be not much prefered by visitiors.*

###**Lets check how room types are divided in each borough**

In [ ]:
#----------------------Block 27----------------------------------

#looking how room_types are divided in each borough
roomtype_in_borough = airbnbNY_data.groupby(['borough','room_type']).agg({'room_type':'count'}) 
roomtype_in_borough = roomtype_in_borough.unstack()  #to change wide table to a long table
roomtype_in_borough

In [ ]:
#----------------------Block 28----------------------------------

roomtype_in_borough.columns  #checking for columns of above dataframe

In [ ]:
#----------------------Block 29----------------------------------

#plotting the pie charts for each borough to get the percentage split visualization of different room types 
plt.rcParams['figure.figsize'] = (8, 15)  #defining figure
fig, axes = plt.subplots(3,2)  #creating figure and multiple axes(3*2=6)
fig.suptitle('Percentage split of each room type in each borough')  #to give a main title for all plots of 6 axes
label=[]
k=0
for i in range(3):
  label.append(roomtype_in_borough.columns[i][1])  #storing different room types in the list
for i in range(3):
  for j in range(2):
    if i == 2 and j==1:
      break
    axes[i,j].pie(roomtype_in_borough.iloc[k],labels=label, autopct='%1.1f%%',radius=1)  #plotting pie chart for each borough in different axes
    axes[i,j].set_title(list(roomtype_in_borough.index)[k])  #setting title as the name of borough for each pie plot 
    k=k+1
plt.figlegend(label)  #to place a legend(room type) in the figure
axes[2][1].set_axis_off()  #to turn off the axes[2][1]


*In each borough shared room is very negligent.*

*One thing noticable is that among all boroughs only Manhattan has more number of entire home than private rooms. Other than Manhattan in each borough private room takes up that place.*

###**Lets check on how does prices distributed among these room types**

In [ ]:
#----------------------Block 30----------------------------------

plt.rcParams['figure.figsize'] = (8, 6)
sns.boxplot(x='room_type', y='price', data = airbnbNY_data)  #box plot showing distribution of price for each room type
plt.title('Box plot for the prices of room type',fontsize=15)
plt.show()

*Prices for each room type is strongly influenced by the outliers.*

*For each roomtype maximum price saturates at nearby 300-400 and above that mostly are outliers.*

In [ ]:
#----------------------Block 31----------------------------------

price_df = airbnbNY_data[airbnbNY_data['price']<600] #sub dataframe to treat outliers
sns.boxplot(x='room_type', y='price', data = price_df) 
plt.title('Box plot for the prices of room type after treating outliers',fontsize=15)
plt.show()

*Note :  Although we have removed whole lot outliers but some are still there.*

*For entire apartments price range is varying till 370 approx and a median or an average value somewhere lies nearby 140-150.*

*For private room price range is varying till 160-170 approx rest are still  outliers and a median or an average value somewhere lies nearby 70-80.*

*For shared rooms price range is varying till 120 approx and a median or an average value somewhere lies nearby 40-50.*


###**Now lets see whether every borough have a nearby same average price range or it differs**

In [ ]:
#----------------------Block 32----------------------------------

#average price of each roomtype in each borough
#taken a subdataframe with removed outliers so that outliers doesn't affect the average price much 
average_price = price_df.groupby(['borough','room_type']).agg({'price':'mean'}).reset_index() 
average_price

In [ ]:
#----------------------Block 33----------------------------------

#visualising Block 32
plt.figure(figsize=(20,8))
#to create a Sub dataframe from average price dataframe where room type is entire room
entire_home = average_price[average_price['room_type']=='Entire home/apt']
#to create a Sub dataframe from average price dataframe where room type is private room
private_room = average_price[average_price['room_type']=='Private room']
#to create a Sub dataframe from average price dataframe where room type is shared room
shared_room =  average_price[average_price['room_type']=='Shared room'] 
x_axis=np.arange(len(entire_home['borough']))  #to get evenly spaced values within range of len(entire_home['borough'])
plt.bar(x_axis-0.3,entire_home['price'],label ='Entire Home',color='wheat',width=0.3)  #to get first bar stick of group of three bars
plt.bar(x_axis,private_room['price'],label ='Private Room',color='goldenrod',width=0.3)  #to get second bar stick of group of three bars
plt.bar(x_axis+0.3,shared_room['price'],label ='Shared Room',color='gold',width=0.3)  #to get third bar stick of group of three bars
plt.legend()  #to place legend in the figure
plt.xticks(x_axis,entire_home['borough'])
plt.xlabel('Boroughs',fontsize=13)
plt.ylabel('Average price',fontsize=13)
plt.title("Average Price of each room type in each borough",fontsize=15)
plt.show()

*Manhattan is leading in a scoreboard of higher prices and Brooklyn follows Manhattan.*

*Manhattan and Brooklyn are tourist areas in NYC and as everywhere rule higher the tourism, higher the prices of rentals also applies here.*  

###**Room Type and number of reviews**

In [ ]:
#----------------------Block 34----------------------------------

#to find no. of reviews for each room type
roomtype_reviews = airbnbNY_data.groupby('room_type').agg({'number_of_reviews':'sum'}) #grouping on room type and adding the number of reviews then
plt.rcParams['figure.figsize']=(10,6)
roomtype_reviews.plot.bar(color='brown')  #bar plot representing no. of reviews for each room type
plt.ylabel('Total number of reviews')
plt.title('Total number of reviews each room type have',fontsize=15)
plt.show()

*Number of reviews follow the same pattern as of the availabilities of each room type in listings. We have most no. of entire homes in our listings and so do reviews for them are most. It goes in same fashion for private and shared room as well.*  

###**After analysis of various factors about roomtype we have gathered following info :**

*1. Entire homes are one of the most available rental in New York City whereas shared room are very less preferable.* 

*2. In every borough private rooms are more in counts whereas in Manhattan entire home wins this race.*

*3. For entire homes nearby 75% of rentals are under the price of 200 and in private room its somewhere near 100 and for shared ones its 75 somewhere.*

*4. Manhattan is most expensive borough in each of the room types, Brooklyn follows Manhattan, then Queens and Staten Island and Bronx are somewhere at same place.*



##***3. Visualising and analysing about hosts***

In [ ]:
#---------------------Block 35----------------------------

#Checking for unique values in host id
airbnbNY_data['host_id'].nunique()

*We have 37457 unique hosts in our data. Rest are duplicates and have appeared multiple times in data.* 

*Calculated host listing counts means no. of listings specific host have in our data. For example say host id 2009___ something have calculated host listing value 6 then that host id has 6 listings in our data. We'll verify that as well.*

In [ ]:
#---------------------Block 36----------------------------

#sum of unique host listing count for unique host ids
x = airbnbNY_data.groupby('host_id').agg({'calculated_host_listings_count':'unique'}).values
x=x.reshape(x.shape[0],)
sum(list(x))

In [ ]:
#---------------------Block 37----------------------------

#Just picking up a row to get host_id and its respective host listing counts
airbnbNY_data.iloc[0:1]

*So the host id 2787 has calculated host listing count value as 6. Lets check if thats true.* 

In [ ]:
#---------------------Block 38----------------------------

#Selecting all the observations with host id 2787
airbnbNY_data[airbnbNY_data['host_id']==2787]

*So the host has total of 6 listings in Brooklyn borough.*

###**Let's look for some of the top and busiest host in our dataset**

*Note : We have considered here the host to be top or busiest if they have more number of rentals.*

In [ ]:
#---------------------Block 39----------------------------

#grouping by host_id and then counting the ids to know the no. of times host id has appeared
top_host = airbnbNY_data.groupby('host_id').agg({'id':'count'}).sort_values(by = 'id', ascending = False)[0:10] 
top_host.rename(columns = {'id':'hostings_count'},inplace =True)  #renaming the column 'id' with hosting counts
plt.rcParams['figure.figsize']=(10,5)
top_host.plot.bar(color='darkseagreen')  #bar plot representing no. of hostings across top 10 hosts
plt.ylabel('Number of hostings',fontsize=13)
plt.title('Top 10 hosts with most number of hostings',fontsize=15)  #title for bar plot
for i,host_count in enumerate(list(top_host['hostings_count'])):
  plt.text(x=i, y = host_count+3, s= str(host_count))  #to put text over bar sticks


*There is a steep decrease in each for top 3 host counts and also they have too many hostings in it.* 

*Now we'll try to dig a bit deep on these hosts*


###**Lets look for how calculated host listing is distributed**

In [ ]:
#-------------------------block 39-a---------------------------------

plt.rcParams['figure.figsize']=(12,8)
sns.distplot(airbnbNY_data['calculated_host_listings_count'],color='goldenrod') #distribution plot for calculated host listing count
plt.title('Distribution of Host listing count',fontsize=15)
plt.show()

In [ ]:
#-------------------------block 39-b---------------------------------

#distribution plot for calculated host listing count with calculated host listing count<10
sns.distplot(airbnbNY_data[airbnbNY_data['calculated_host_listings_count']<10]['calculated_host_listings_count'],color='darkgoldenrod') 
plt.title('Distribution of Host listing count with host listing count less than 10',fontsize=15)
plt.show()

*From above two blocks we came to know that having a host listing counts of 327 or so and even having a host lising count greater than one is rare in our dataset. Most host has only 1 rentals thus a value of calculated host listing count as 1.*

In [ ]:
#---------------------Block 40----------------------------

#function to create a sub dataframe only with these top 10 hosts 
def host_selector(col):
  if col in top_host.index:
    return True
  else:
    return False

In [ ]:
#---------------------Block 41----------------------------

#applying previous defined function in block 40 to create a sub dataframe with name tophost_df 
tophost_df = airbnbNY_data[airbnbNY_data['host_id'].apply(host_selector)]
tophost_df['host_id'].value_counts()

### **Let's see in which borough/neighbourhood these top 10 hosts have most of their hostings,room type offered by these hosts and price distribution for these hosts**


In [ ]:
#---------------------Block 42----------------------------

tophost_df.groupby('host_id')['borough'].value_counts()  #checking for no. of hostings in different boroughs by top 10 hosts

*Manhattan is a winning borough in this case if you see, out of the 10 hosts 7 have all their hostings in Manhattan whereas for rest 3 most of their hostings in Manhattan except host id 137358866.*



*Now we have seen that most of them belongs to Mahnhattan borough.*

**Now lets look for the neighbourhoods that these hosts occupied**

In [ ]:
#---------------------Block 43----------------------------

#Checking for no. of hostings in different neighbourhoods by these top 10 hosts
tophost_nghbrhood = tophost_df.groupby(['borough','neighbourhood']).agg({'id':'count'}).sort_values(by = 'id',ascending =False).reset_index()
tophost_nghbrhood.rename(columns={'id':'count'}, inplace = True)  #renaming the column names
unique_borough= set(list(tophost_nghbrhood['borough']))  #to store unique boroughs of tophost_nghbrhood dataframe
borough_dict ={}  #initialising an empty dictionary
index = 0 
color = ['cyan','purple','red','orange','green']
for borough in unique_borough:
  borough_dict[borough] = color[index]  #storing boroughs as key and color as value in the dictionary
  index = index +1
color_list=[] 
for borough in tophost_nghbrhood['borough']:
  color_list.append(borough_dict[borough])  #storing different colors in the list according to the borough
plt.figure(figsize=(25,10))
plt.bar(tophost_nghbrhood['neighbourhood'], tophost_nghbrhood['count'], color = color_list) 
labels = list(borough_dict.keys())
#to get labelling on plot
custom_lines = [Line2D([0], [0], color='cyan', lw=4), 
                Line2D([0], [0], color='purple', lw=4),
                Line2D([0], [0], color='red', lw=4),
                Line2D([0], [0], color='orange', lw=4),
                Line2D([0], [0], color='green', lw=4)]
plt.xticks(rotation = 90)  #to rotate x-ticks by 90 degree
plt.legend(custom_lines,labels)  #to get legend
plt.xlabel('neighbourhood',fontsize=13)
plt.ylabel('Number of hostings',fontsize=13)
plt.title("Neighbourhoods in which top 10 hosts have their hostings",fontsize=15)
plt.show()

*Top and major neighbourhoods comes under Manhattan and are the major attraction of tourists. That's the reason of multiple hostings are there in these neighbourhood.*

In [ ]:
#---------------------Block 44----------------------------

#checking for no. of hostings for each room type by top 10 hosts
tophost_df.groupby(['host_id','room_type']).agg({'id':'count'}) 

*Ok so mostly they offer entire home only with just few of them offering private rooms and no shared rooms*

**Lets check on how the price of these hosts differs from the average price of boroughs. As most of them only have entire homes only so will check for that only**

In [ ]:
#---------------------Block 45----------------------------

avgprice = tophost_df.groupby(['host_id','room_type']).agg({'price':'mean'}).reset_index()  #average price for each room type across top 10 hosts
fig = plt.figure(figsize=(10,6))
#creating a subdataframe from avgprice dataframe where room type is entire home
avgprice = avgprice[avgprice['room_type']=='Entire home/apt'].reset_index().sort_values(by = 'price',ascending=False) 
ax = fig.gca() #to get current axes
avgprice.plot.bar(x='host_id', y='price', ax = ax,color='olive')  #bar plot representing average price for entire home across each host among top 10 hosts
plt.title('average prices across top 10 hosts for entire home roomtype',fontsize=13)
for i,price in enumerate(list(avgprice['price'])):
  ax.text(x=i,y = price+5, s = str(round(price,2)))

*Initially in block 31 when we plotted the box plot we saw that the median or an average price of entire home lies somewhere near 160-170 and also from block 33 we can see that the average entire home prices in Manhattan is 210 as most of these host belongs to Manhattan only.* 

*But there we have removed the outliers by setting a price threshold with 600 so lets try that here as well to see actual price differences among other hosts and these hosts.*

In [ ]:
#---------------------Block 46----------------------------

#applying the same thing as in block 45 but setting price threshold as 350
price_350 = tophost_df[tophost_df['price']<600]  #to create a subdataframe from tophost_df where price <600
avgprice = price_350.groupby(['host_id','room_type']).agg({'price':'mean'}).reset_index()
avgprice = avgprice[avgprice['room_type']=='Entire home/apt'].reset_index().sort_values(by = 'price',ascending=False)
fig = plt.figure(figsize=(10,6))
ax = fig.gca()
avgprice.plot.bar(x='host_id', y='price', ax = ax,color='sandybrown')  #bar plot representing average price for entire home across top 10 hosts
plt.title('average prices across top 10 hosts for entire home roomtype',fontsize=13)
for i,price in enumerate(list(avgprice['price'])):
  ax.text(x=i,y = price+5, s = str(round(price,2)))

*So even after setting a threshold as 600 for price, there is not much change in average prices and nearly these appears to be higher than average prices we have seen in block 33.*

*So we can draw following conclusions. - Generally most of these top hosts might have luxurious apartments and most of them some where have a charge higher than the average charges we have seen earlier in block 31 and 33.*


###**So collected info about hosts are :**

*1. We have 37k + unique hosts in our dataset.*

*2. Most hosts has single rentals only. Although there are also such host having rentals as high as 327, then in 200s, 100s and 50s but general population in our dataset has 1 only.*

*3. All these top host mostly belongs to Manhattan. Out of total rentals that these top 10 hosts combinely have only 88 from them is in other boroughs rest all are in Manhattan.*

*4. Financial district neighbourhood is the one where all these top host mostly have their rentals.*

*5. When trying to check, is it the price that sets these host difference it actually came into notice that these host in general have higher price than the average price in their respective boroughs. So it might following a phenomenon of brand value as we all know the higher the brand value higher the price. Yeah and also along with some good facilities.*